## GIS for Climate Action - VRP exercise using Python API

This notebook solves VRP for a Farm Collective that delivers fresh produce from the farms in Canterbury, New Zealand directly to residents in the nearby city of Christchurch and surrounding towns. Here are the steps  to solve the problem with inputs to get routes that can be opened in Navigator or other navigation app. This notebook uses ArcGIS API for Python for automating the workflow.


### 1. Import required python libraries

In [ ]:
import arcgis
from arcgis.gis import GIS
import pandas as pd
import datetime
from arcgis import geocoding
from arcgis.features import Feature, FeatureSet

### 2. Authenticate the connection
A few other ways of authentication : https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/

In [ ]:
my_gis = GIS("https://geosaurus.maps.arcgis.com/home/index.html", "arcgis_python", "amazing_arcgis_123") 
    
print("Logged in as anonymous user to " + my_gis.properties.portalName)
print("Logged in as " + str(my_gis.properties.user.username))

Logged in as anonymous user to ArcGIS Online
Logged in as arcgis_python


### 3. Create Orders Layer with Address Information

In [ ]:
orders_csv = "Addresses.csv"

# Read the csv file and convert the data to feature set
orders_df = pd.read_csv(orders_csv)
orders_sdf = pd.DataFrame.spatial.from_df(orders_df, "Address")
orders_fset = orders_sdf.spatial.to_featureset()
orders_fset

FileNotFoundError: [Errno 2] No such file or directory: 'Addresses.csv'

### 4. Create Routes Layer

In [ ]:
routes_csv = "routes.csv"

# Read the csv file
routes_df = pd.read_csv(routes_csv, parse_dates=["EarliestStartTime", "LatestStartTime"], date_parser=pd.to_datetime)
routes_df["EarliestStartTime"] = routes_df["EarliestStartTime"].astype("int64") / 10 ** 6
routes_df["LatestStartTime"] = routes_df["LatestStartTime"].astype("int64") / 10 ** 6
routes_fset = arcgis.features.FeatureSet.from_dataframe(routes_df)
routes_df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'routes.csv'

### 5. Create Depots Layer with an Address

In [ ]:
depot_geocoded_fs = geocoding.geocode("5110 Arundel Rakaia Gorge Road, Alford Forest, Mount Somers, Canterbury, 7771", 
                                      as_featureset=True, max_locations=1)
depot_geocoded_fs.features[0].attributes["Name"] = "5110 Arundel Rakaia Gorge Road, Alford Forest, Mount Somers, Canterbury, 7771"
depot_geocoded_fs.features
depots_fset = depot_geocoded_fs.sdf.spatial.to_featureset()
depot_geocoded_fs

### 6. Draw the Depots and Orders in map

In [ ]:
# Create a map instance to visualize inputs in map
map_view_inputs = my_gis.map('Christchurch, New Zealand', zoomlevel=8)
map_view_inputs

In [ ]:
# Visualize order and depot locations with symbology
map_view_inputs.draw(orders_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","color": [76,115,0,255],"size": 8})
map_view_inputs.draw(depot_geocoded_fs, symbol={"type": "esriSMS","style": "esriSMSSquare","color": [255,115,0,255], "size": 10})

### 7. Solve VRP

In [ ]:
%%time
today = datetime.datetime.now()
from arcgis.network.analysis import solve_vehicle_routing_problem
results = solve_vehicle_routing_problem(orders= orders_fset,
                                        depots = depots_fset,
                                        routes = routes_fset, 
                                        save_route_data='true',
                                        populate_directions='true',
                                        travel_mode="Driving Time",
                                        default_date=today)

print('Analysis succeeded? {}'.format(results.solve_succeeded))

### 8. Look into the Results

In [ ]:
# Display the output routes in a pandas dataframe.
out_routes_df = results.out_routes.sdf
out_routes_df[['Name','OrderCount','StartTime','EndTime','TotalCost','TotalDistance','TotalTime','TotalTravelTime','StartTimeUTC','EndTimeUTC']]

In [ ]:
out_stops_df = results.out_stops.sdf
out_stops_df[['Name','RouteName','Sequence','ArriveTime','DepartTime']].sort_values(by=['RouteName',
                                                                                       'Sequence'])

### 9. Display Output Routes

In [ ]:
# Create a map instance to visualize inputs in map
map_view_outputs = my_gis.map('Christchurch, New Zealand', zoomlevel=8)
map_view_outputs

In [ ]:
#Visusalize the inputsn with different symbols
map_view_outputs.draw(orders_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSCircle",
                                           "color": [76,115,0,255],"size": 8})
map_view_outputs.draw(depots_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSSquare",
                                           "color": [255,115,0,255], "size": 10})

#Visualize the first route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[0])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [0,100,240,255],"size":10})

#Visualize the second route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[1])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [255,0,0,255],"size":10})

### 10. Create Route Layers for Navigation

In [ ]:
route_data = results.out_route_data.download('.')
route_data_item = my_gis.content.add({"type": "File Geodatabase"}, route_data)
route_layers = arcgis.features.analysis.create_route_layers(route_data_item, 
                                                            delete_route_data_item=True)
for route_layer in route_layers:
    route_layer.share(org=True)
    display(route_layer.homepage)
    display(route_layer)

NameError: name 'results' is not defined

### 11. Conclusion
The routes are ready for navigation. In this way, you can convert a workflow into a python Script to solve VRP.